In [1]:
import pandas as pd
import numpy as np
import math

In [5]:
class TreeNode:
    def __init__(self, data, attributes=None):
        self.data = data  # value of the node
        self.attributes = attributes if attributes is not None else {}  # attributes in dict
        self.children = []  # list of child nodes

    def add_child(self, child_node):
        self.children.append(child_node)

def find_depth(node):
    # Base case: If the node has no children, its depth is 0
    if not node.children:
        return 0
    else:
        # Find the depth of each child node recursively
        child_depths = [find_depth(child) for child in node.children]
        # Return the maximum depth among child nodes + 1 (for the current node)
        return max(child_depths) + 1

# Example usage:
root = TreeNode("Root")
child1 = TreeNode("Child 1")
child2 = TreeNode("Child 2")
grandchild1 = TreeNode("Grandchild 1")
grandchild2 = TreeNode("Grandchild 2")

root.add_child(child1)
root.add_child(child2)
child1.add_child(grandchild1)
child2.add_child(grandchild2)

# Calculate the depth of the root node
root_depth = find_depth(root)
print("Depth of the root node:", root_depth)  # Should print 2


Depth of the root node: 2
